<a href="https://colab.research.google.com/github/Leo777Deo/CFD_Project/blob/main/alternative.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# CASE 2:

# Make all inlet and outlet as transfinite lines

from os import write
import numpy as np
import matplotlib.pyplot as plt

# Define bump parameters
delta = np.pi / 28
K = 1.3
c = 0
yp = 0

# Define the function for the top boundary
def f_top(x):
    return np.sqrt(((x**2 * (np.tan(delta))**2) + c) / (1 / np.cos(np.arctan(yp / K)))**2) * np.sin(x) * np.sin((yp + np.pi) / 2)

# Discretize the function into points
N_points_bump = 50 # Number of points for bump
x_vals = np.linspace(0, np.pi, N_points_bump)
y_vals_top = f_top(x_vals)

# Define the starting point of the new line

Delta_x = 6 # Distance in terms of discretization points from the maximum of the bump
Delta_conv = 1 # Distance in terms of discretization points from the start of the inner part of conv. to outer
y_dist = 3 # multiplication factor that increases distance between max of bump and initial point of inner convergent
thick_conv = 0.02 # Additive quantity to the end point of the convergent to close it
thick_conv2 = 0.45 * thick_conv
angle_deg = -5 # Angle of the convergent, taken from Web Plot Digitizer from image of the thesis of sweden guy
index_line_bump = np.argmax(y_vals_top) + Delta_x
x_start, y_start = x_vals[index_line_bump], y_dist * max(y_vals_top)

x_start_up, y_start_up = x_vals[index_line_bump], (y_dist * max(y_vals_top)) + thick_conv
angle_rad = np.radians(angle_deg)

x_start_conv = x_vals[index_line_bump - 1]
y_start_conv = y_start_up - (x_start - x_start_conv)*np.tan(angle_rad)


# Define the length of the line segment
End_line_param = x_vals[-1] + 4 - x_vals[-1]
line_length = End_line_param - x_start
x_end = x_start + line_length * np.cos(angle_rad)
y_end = y_start + line_length * np.sin(angle_rad)

x_end_up = x_end
y_end_up = y_start_up + line_length * np.sin(angle_rad)

# Define Far Field
Y_DIST = 150 # Multiplication factor of maximum of bump to create far field Y
Y_END = Y_DIST * y_vals_top[np.argmax(y_vals_top)]
X_DIST = -25 # Multiplication factor of end x point of bump to create far field X
X_START = X_DIST * x_vals[np.argmax(x_vals)]
X_END = x_end


size_bl = 0.00001
n_bl = 65 # Number of layers  # Number of layers
r = 1.1  # Growth ratio

# Compute the total boundary layer thickness
thick_bl = size_bl * (r**n_bl - 1) / (r - 1)

thick_FF = thick_bl

# .geo file creation:
with open("Bumpconv.geo", "w") as file:

    file.write("SetFactory('OpenCASCADE');\n")

    file.write(f"SIZE_BL= {size_bl};\n")
    file.write(f"THICK_BL= {thick_bl};\n")

    file.write("h = 0.01;\n") # Size of cells close to bump and convergent
    file.write("H = 1;\n") # Size of cells close to farfield
    file.write(f"H_BL = {thick_bl};\n") # Size of geometrical step of structured mesh

    # Write points for the bump
    for i, (x, y) in enumerate(zip(x_vals, y_vals_top)):
      file.write(f"Point({i + 1}) = {{{x}, {y}, 0, h}};\n")

# Define spline from the first point to the maximum
    file.write(f"Spline(1) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top) + 1))}}};\n")

# Define spline from the maximum to the end
    file.write(f"Spline(2) = {{{', '.join(str(i + 1) for i in range(np.argmax(y_vals_top), len(x_vals)))}")
    file.write("};\n")

    # Define points for the new line with -5° slope and the upper parallel line
    new_point_start = len(x_vals) + 1
    new_point_end = new_point_start + 1
    upper_point_start = new_point_end + 1
    upper_point_end = upper_point_start + 1
    upper_conv = upper_point_end + 1
    lower_conv = upper_conv + 1
    circle_centre = lower_conv + 1

    # Write the convex shape points and lines
    file.write(f"Point({new_point_start}) = {{{x_start}, {y_start}, 0, h}};\n")
    file.write(f"Point({new_point_end}) = {{{x_end}, {y_end}, 0, h}};\n")
    file.write(f"Line({len(x_vals) + 1}) = {{{new_point_start}, {new_point_end}}};\n")

    file.write(f"Point({upper_point_start}) = {{{x_start_up}, {y_start_up}, 0, h}};\n")
    file.write(f"Point({upper_point_end}) = {{{x_end_up}, {y_end_up}, 0, h}};\n")
    file.write(f"Line({len(x_vals) + 2}) = {{{upper_point_end}, {upper_point_start}}};\n")

    file.write(f"Point({upper_conv}) = {{{x_start_conv}, {y_start_conv - thick_conv2}, 0, h}};\n")
    file.write(f"Point({lower_conv}) = {{{x_start_conv}, {y_start_conv - thick_conv2 - (0.5*thick_conv - thick_conv2)}, 0, h}};\n")
    file.write(f"Point({circle_centre}) = {{{x_start_conv}, {y_start_conv - thick_conv2 - (0.5*thick_conv - thick_conv2)/2}, 0, h}};\n")

    # Define connecting lines for closed convex surface
    file.write(f"Line({len(x_vals) + 3}) = {{{lower_conv}, {new_point_start}}};\n")
    file.write(f"Line({len(x_vals) + 4}) = {{{upper_point_start}, {upper_conv}}};\n")
    file.write(f"Circle({len(x_vals) + 14}) = {{{lower_conv}, {circle_centre}, {upper_conv}}};\n")  # Circle definition (start point, centre point, end point)
    file.write(f"Line({len(x_vals) + 15}) = {{{new_point_end}, {upper_point_end}}};\n")

    # Boundary points for the entire domain
    boundary_point_1 = circle_centre + 1
    boundary_point_2 = boundary_point_1 + 1
    boundary_point_3 = boundary_point_2 + 1
    boundary_point_4 = boundary_point_3 + 1
    boundary_point_5 = boundary_point_4 + 1

    boundary_point_6 = boundary_point_5 + 1
    boundary_point_7 = boundary_point_6 + 1
    boundary_point_8 = boundary_point_7 + 1

    file.write(f"Point({boundary_point_1}) = {{{x_end}, 0, 0, h}};\n")
    file.write(f"Point({boundary_point_2}) = {{{X_END}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_3}) = {{{X_START}, {Y_END}, 0, H}};\n")
    file.write(f"Point({boundary_point_4}) = {{{X_START}, 0, 0, H}};\n")

    file.write(f"Point({boundary_point_5}) = {{{X_START}, {thick_FF}, 0, H}};\n") # thick_bl
    file.write(f"Point({boundary_point_6}) = {{{X_END}, {thick_FF}, 0, H}};\n") # thick_bl
    file.write(f"Point({boundary_point_7}) = {{{X_END}, {y_end - thick_FF}, 0, H}};\n") # thick_bl
    file.write(f"Point({boundary_point_8}) = {{{X_END}, {y_end_up + thick_FF}, 0, H}};\n") # thick_bl

    # Define lines for the main boundary
    file.write(f"Line({len(x_vals) + 5}) = {{{boundary_point_1}, {boundary_point_6}}};\n")
    file.write(f"Line({len(x_vals) + 6}) = {{{boundary_point_6}, {boundary_point_7}}};\n")
    file.write(f"Line({len(x_vals) + 16}) = {{{boundary_point_7}, {new_point_end}}};\n")
    file.write(f"Line({len(x_vals) + 7}) = {{{new_point_end}, {upper_point_end}")
    file.write("};\n")
    file.write(f"Line({len(x_vals) + 8}) = {{{upper_point_end}, {boundary_point_8}}};\n")
    file.write(f"Line({len(x_vals) + 17}) = {{{boundary_point_8}, {boundary_point_2}}};\n")
    file.write(f"Line({len(x_vals) + 9}) = {{{boundary_point_2}, {boundary_point_3}}};\n")
    file.write(f"Line({len(x_vals) + 10}) = {{{boundary_point_3}, {boundary_point_5}}};\n")
    file.write(f"Line({len(x_vals) + 11}) = {{{boundary_point_5}, {boundary_point_4}}};\n")
    file.write(f"Line({len(x_vals) + 12}) = {{{boundary_point_4}, 1}};\n")  # Connect back to start of bump
    file.write(f"Line({len(x_vals) + 13}) = {{{new_point_start - 1},{boundary_point_1} }};\n")


    file.write(f"Transfinite Line{{{len(x_vals) + 5}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 11}}} = {n_bl} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 16}}} = {n_bl} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 8}}} = {n_bl} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 6}}} = {150} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 17}}} = {150} Using Progression {r};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 10}}} = {150} Using Progression {1/r};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 12}}} = {150} Using Progression {1};\n")
    file.write(f"Transfinite Line{{{len(x_vals) + 13}}} = {150} Using Progression {1};\n")
    #file.write(f"Transfinite Line{{{len(x_vals) + 14}}} = {10} Using Progression {1};\n")

    # Define a single Line Loop for the entire domain boundary (the outermost loop)
    file.write("Line Loop(1) = {")
    file.write("1, 2") # Spline 1 and 2
    file.write(f",{len(x_vals) + 13}, {len(x_vals) + 5}, {len(x_vals) + 6}, {len(x_vals) + 16} ,{len(x_vals) + 7}, {len(x_vals) + 8}, {len(x_vals) + 17}, {len(x_vals) + 9},{len(x_vals) + 10} ,{len(x_vals) + 11} ,{len(x_vals) + 12}")
    file.write("};\n")

    file.write("Line Loop(2) = {")
    file.write(f"{len(x_vals) + 1}, -{len(x_vals) + 15}, {len(x_vals) + 2}, {len(x_vals) + 4}, {len(x_vals) + 14}, {len(x_vals) + 3}")
    file.write("};\n")

    file.write("Plane Surface(1) = {1, 2};\n") # It tells GMSH to mesh only on 1 and not account for 2

    # Physical Surface definition
    file.write("Physical Surface('VOLUME') = {1};\n")

    # Physical lines and surfaces for boundary conditions
    file.write(f"Physical Line('inlet') = {{{len(x_vals) + 10},{len(x_vals) + 11} }};\n")

    file.write("Physical Line('lower wall') = {")
    file.write(f"{len(x_vals) + 12},")
    file.write("1, 2")
    file.write(f",{len(x_vals) + 13}")
    file.write("};\n")

    file.write(f"Physical Line('upper wall') = {{{len(x_vals) + 8}}};\n")

    file.write("Physical Line('CONVERGENT') = {")
    file.write(f"{len(x_vals) + 1}, -{len(x_vals) + 15}, {len(x_vals) + 2}, {len(x_vals) + 4}, {len(x_vals) + 14}, {len(x_vals) + 3}")
    file.write("};\n")

    file.write(f"Physical Line('outlet') = {{{len(x_vals) + 8}, {len(x_vals) + 17}}};\n")
    file.write(f"Physical Line('ENGINE') = {{{len(x_vals) + 5}, {len(x_vals) + 6}, {len(x_vals) + 16}}};\n")


    # MESH:

    file.write("Field[1] = BoundaryLayer;\n")
    file.write("Field[1].CurvesList = {")
    file.write(f"{len(x_vals) + 12},")
    file.write("1, 2")
    file.write(f",{len(x_vals) + 13}")
    file.write("};\n")
    file.write("Field[1].Quads = 1;\n")
    file.write(f"Field[1].Ratio = {r};\n")
    file.write(f"Field[1].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[1].Thickness = {thick_bl};\n") #  0.07
    file.write("Field[1].FanPointsList = {1};\n")
    file.write("Field[1].FanPointsSizesList = {5};\n")
    file.write("BoundaryLayer Field = 1;\n")

    file.write("Field[2] = BoundaryLayer;\n")
    file.write("Field[2].CurvesList = {")
    file.write(f"{len(x_vals) + 2},")
    file.write(f"{len(x_vals) + 4},")
    file.write(f"{len(x_vals) + 14},")
    file.write(f"{len(x_vals) + 3},")
    file.write(f"{len(x_vals) + 1}")
    file.write("};\n")
    file.write("Field[2].Quads = 1;\n")
    file.write(f"Field[2].Ratio = {r};\n")
    file.write(f"Field[2].Size = {size_bl};\n") #  0.00001
    file.write(f"Field[2].Thickness = {thick_bl};\n") #  0.07
    #file.write("Field[2].FanPointsList = {1};\n")
    #file.write("Field[2].FanPointsSizesList = {5};\n")
    file.write("BoundaryLayer Field = 1;\n")

from google.colab import files
files.download("Bumpconv.geo")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>